# Running MEEP Simulations

[MEEP](https://meep.readthedocs.io/) is an open-source FDTD electromagnetic simulator. This notebook demonstrates using the `gsim.meep` API to run an S-parameter simulation on a photonic Y-branch.

**Requirements:**

- UBC PDK: `uv pip install ubcpdk`
- [GDSFactory+](https://gdsfactory.com) account for cloud simulation

### Load a pcell from UBC PDK

In [ ]:
from ubcpdk import PDK, cells

PDK.activate()

c = cells.ebeam_y_1550()
c

### Configure and run simulation

In [ ]:
from gsim import meep

sim = meep.Simulation()

sim.geometry.component = c
sim.geometry.z_crop = "auto"

sim.materials = {"si": 3.47, "SiO2": 1.44}

sim.source.port = "o1"
sim.source.wavelength = 1.55
sim.source.bandwidth = 0.01
sim.source.num_freqs = 11

sim.monitors = ["o1", "o2", "o3"]

sim.solver.resolution = 20
sim.solver.stopping = meep.DFTDecay(max_time=200, threshold=1e-3, min_time=100)
sim.solver.simplify_tol = 0.01
sim.solver.subpixel = True
sim.solver.save_geometry = True
sim.solver.save_fields = True
sim.solver.save_animation = True
sim.solver.verbose_interval = 5.0

print(sim.validate_config())

In [ ]:
sim.plot_2d(slices="xyz")

### Run simulation on cloud

In [ ]:
# Write config (layout.gds + sim_config.json + run_meep.py)
sim.write_config("./meep-sim-test")

In [ ]:
# Run on GDSFactory+ cloud
result = sim.run("./meep-sim-test")

In [ ]:
result.plot(db=True)

In [ ]:
result.show_diagnostics()